In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
# Since our data is comprised of images, we have to load each image from each directory and store them into arrays
# This function returns all the filenames in the chosen directory
def get_files(directory):
    files = []
    for dirname, _, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    return files

train = []
labels = []

for file in get_files('/kaggle/input/campqmind-hackathon-2022/train/'):
    img = cv2.imread(file)
    img_array = np.asarray(img)
    

    if img_array.shape != (102, 136, 3):
        continue

    
    converted = tf.image.rgb_to_grayscale(img_array)
    
    train.append(converted)
    labels.append(file.split('/')[-2])

test = []
for file in get_files('/kaggle/input/campqmind-hackathon-2022/test/'):
    img = cv2.imread(file)
    img_array = np.asarray(img)
    test.append(img_array)
    

print(len(train), len(labels))

# We have to use a plotting function and cv2 to show the image from the array data

10498 10498


In [21]:
from sklearn import preprocessing

# Encode the labels into integers classes 0-2
le = preprocessing.LabelEncoder()
le.fit(labels)
lab = le.transform(labels)

# Loading Data in

In [22]:
os.listdir('/kaggle')

['lib', 'input', 'working']

In [23]:
os.listdir('/kaggle/input/campqmind-hackathon-2022')

['sample_submission.csv', 'test', 'train']

In [24]:
from sklearn.model_selection import train_test_split

train = np.array(train)
train_data, test_data, train_labels, test_labels = train_test_split(train, lab, test_size=0.33, random_state=42)

print(len(train_data), len(train_labels))
print(len(test_data), len(test_labels))

print(test_labels)

7033 7033
3465 3465
[1 0 0 ... 2 2 0]


In [26]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(102, 136, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3))

model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 100, 134, 32)      320       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 50, 67, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 48, 65, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 24, 32, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 22, 30, 64)        36928     
_________________________________________________________________
flatten_5 (Flatten)          (None, 42240)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)               

In [27]:
test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

109/109 - 7s - loss: 0.3901 - accuracy: 0.9362
